In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, accuracy_score
    )

In [2]:
from google.colab import drive


# 執行階段 - 變更執行階段類型 - GPU
# 12 hours
drive.mount('/content/gdrive')

!ls gdrive/MyDrive/
!nvidia-smi
!python -c "import torch; print(torch.__version__)"

Mounted at /content/gdrive
 01-拍攝檔-347P.rar		    NN
'10 27'				   'NTOU transcript.pdf'
 20210721_093952.jpg		    research
 520.pdf			    Untitled0.ipynb
'Adobe Scan Apr 23, 2021.pdf'	    VRDL
 CACS2019			    中華民國斐陶斐榮譽學會
'CamScanner 12-07-2021 22.04.pdf'   劉達度主任獎學金
 DM				    海洋書卷獎
 InShot_20220107_192507746.mp4	    給我們.pdf
 LINE_MOVIE_1591450082471.mp4
Tue Jan 11 07:36:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 O

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


class pyidata(Dataset):
    def __init__(self, DATA_PATH, is_train, pca=False):
        super().__init__()
        self.is_train = is_train
        self.pca = pca
        if self.is_train:
            tr = pd.read_excel(os.path.join(DATA_PATH, 'tr.xlsx'))
            self.y_tr = tr['outcome'].values
            # get data
            col = tr.columns.to_list()
            del col[-1]
            self.x_tr = tr[col].values


            if self.pca:
                pca = PCA(n_components = 0.98)
                self.x_tr = pca.fit_transform(self.x_tr)

            print('train x shape:', self.x_tr.shape)
            print('train y shape:', self.y_tr.shape)
            
            self.x_tr = torch.tensor(self.x_tr)
            self.y_tr = torch.tensor(self.y_tr)

        if not self.is_train: 
            self.te = pd.read_excel(os.path.join(DATA_PATH, 'test_final.xlsx'))# get labels 
            self.te = self.te.values
            if self.pca:
                pca = PCA(n_components = 0.98)
                self.te = pca.fit_transform(self.te)

            print('test x shape:', self.te.shape)
            self.te = torch.tensor(self.te)
            
    def __getitem__(self, index):
        if self.is_train:
            data = self.x_tr[index].unsqueeze(0)
            label = self.y_tr[index]
            return data, label
        if not self.is_train:
            data = self.te[index].unsqueeze(0)
            label = torch.tensor(2)
            return data, label
    def __len__(self):
        if self.is_train:
            return self.x_tr.shape[0]
        if not self.is_train:
            return self.te.shape[0]

In [6]:
# this function aims to measure some metrics
def metric(clf, x, y, mode=None, return_output=False):  
    
    preds = clf.predict(x)
    prec = precision_score(y, preds)
    reca = recall_score(y, preds)
    roc = roc_auc_score(y, preds)
    f1 = f1_score(y, preds)
    acc = accuracy_score(y, preds)
    conf_mat = confusion_matrix(y, preds)
    if mode is not None:
        print(f'=============={mode}=============')
        print("Precision: ", prec)
        print("Recall: ", reca)
        print("ROC score: ", roc)
        print("F1 score: ", f1)
        print("Accuracy score: ", acc)
        print("confusion matrix")
        print(conf_mat)
    
    if return_output is True:
        return [prec, reca, roc, f1, acc]

In [7]:
def train(model, train_loader, val_loader, optimizer, EPOCHS, MODELPATH):
    '''
    training procedure based on cross entropy loss(or your preferable loss)
    *input:
    model: model
    optimizer: SGD or Adam or st else
    scheduler: make learning rate designable, for eg, ExponentialLR
    criterion:default is cross entropy loss
    train_loader: training set
    val_loader: validation set
    EPOCHS: total epochs
    MODELPATH: the place where you save your model
    '''
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    criterion = nn.CrossEntropyLoss()
    
    optimizer = optimizer
    # scheduler = scheduler
    
    model.to(device)
    model.train()
    
    best_acc = 0
    for epoch in range(EPOCHS):
        epoch_loss = 0
        train_acc=0
        num = 0
        for x, y in tqdm(train_loader):
            x = x.to(device).float()
            y = y.to(device)
            
            out = model(x)
            
            #calculate loss part
            try:
                loss = criterion(out, y.long())
            except:
                out = out[0]
                loss = criterion(out, y.long())
                
            epoch_loss += loss.item()
            num += y.size(0)
            
            #calculate train acc
            pred_ind = torch.max(out, 1).indices.view(-1)
            train_acc += (pred_ind == y).sum().item()
            
            #backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        epoch_loss = epoch_loss / num
        train_acc /= num
        test_acc, test_loss = evaluate(model, val_loader)

        if (epoch + 1) % 1 == 0:
            print(f'epoch {epoch+1}%%%%%%%%%%%%%%%%%%%%')
            print(f"train set===>loss: {epoch_loss:.4f}   acc: {100* train_acc:.2f}%")
            print(f"val set=====>loss: {test_loss:.4f}   acc: {100* test_acc:.2f}%")
            print()

        if test_acc > best_acc:
            best_acc = test_acc
            print('save model')
            torch.save(model.state_dict(), MODELPATH)
    print('final model prediction performance on val:', best_acc)
        
        

def evaluate(model, data_loader):
    '''
    loss:cross entropy
    this function aims to evalute the model performance wrt a specific data loader
    *input: 
    model, data loader
    *output:
    accuracy, loss
    '''
    criterion = nn.CrossEntropyLoss()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)
    model.eval()
    acc = 0
    num = 0
    total_loss = 0
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device).float(), y.to(device)
            out = model(x)
            
            loss = criterion(out, y.long())
                
            total_loss += loss.item()
            num += y.size(0)
            
            pred_ind = torch.max(out, 1).indices.view(-1)
            acc += (pred_ind == y).sum().item()
            
        acc /= num
        total_loss = total_loss/num
        return acc, total_loss
def inference(model, weights_path, data_loader):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)
    model.load_state_dict(torch.load(weights_path))

    model.eval()
    num = 0
    preds = torch.empty(0, device=device)
    ground = torch.empty(0, device=device)
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device).float(), y.to(device)
            out = model(x)
            
            num += y.size(0)
            
            pred_ind = torch.max(out, 1).indices.view(-1)

            preds = torch.concat((preds, pred_ind), 0)
            ground = torch.concat((ground, y), 0) 

    preds = preds.cpu().detach().numpy()
    ground = ground.cpu().detach().numpy()

    prec = precision_score(ground, preds)
    reca = recall_score(ground, preds)
    roc = roc_auc_score(ground, preds)
    f1 = f1_score(ground, preds)
    acc = accuracy_score(ground, preds)

    return [prec, reca, roc, f1, acc]




In [8]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, f1=16, f2=32, ks=3):
        super(Net, self).__init__()
        self.model_layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=f1, kernel_size=ks),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(in_channels=f1, out_channels=f2, kernel_size=ks),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),

            nn.AdaptiveAvgPool1d(4),
            nn.Flatten(),

            nn.Linear(f2 * 4, 2)
        )
    def forward(self, x):
        return self.model_layers(x)
model = Net()
model(torch.randn(2, 1, 50)).shape

torch.Size([2, 2])

# smote

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DM/smote_dataset'
trainset = pyidata(DATA_PATH, True)
testset = pyidata(DATA_PATH, False)
le = len(trainset)
start = int(0.8*le)
end = le - start
trainset, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

train_loader = DataLoader(trainset,
              shuffle=True,
              batch_size=32,
              num_workers=4,
              pin_memory=True)
val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)
test_loader = DataLoader(testset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)

train x shape: (82044, 50)
train y shape: (82044,)
test x shape: (979, 51)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

EPOCHS = 700
LR = 0.01
CONV = {
    'f1':32,
    'f2':64,
    'ks':6
}

model_smote = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_smote.parameters()), lr=LR, momentum=0.9)
CONFIG = {
    'model': model_smote,
    'train_loader':train_loader,
    'val_loader':val_loader, 
    'optimizer':optimizer, 
    'EPOCHS':EPOCHS, 
    'MODELPATH':f'/content/drive/MyDrive/DM/CNN/smote_{LR}_{CONV["f1"]}_{CONV["f2"]}_{CONV["ks"]}.pth'
}
train(**CONFIG)

  0%|          | 0/2052 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 2052/2052 [00:07<00:00, 261.53it/s]


epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0204   acc: 61.41%
val set=====>loss: 0.0192   acc: 66.41%

save model


100%|██████████| 2052/2052 [00:07<00:00, 267.24it/s]


epoch 2%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0182   acc: 68.58%
val set=====>loss: 0.0173   acc: 70.10%

save model


100%|██████████| 2052/2052 [00:09<00:00, 210.63it/s]


...

epoch 698%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0014   acc: 98.65%
val set=====>loss: 0.0060   acc: 97.59%



100%|██████████| 2052/2052 [00:08<00:00, 242.66it/s]


epoch 699%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0024   acc: 97.79%
val set=====>loss: 0.0071   acc: 96.35%



100%|██████████| 2052/2052 [00:08<00:00, 242.30it/s]


epoch 700%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0016   acc: 98.40%
val set=====>loss: 0.0057   acc: 96.73%

final model prediction performance on val: 0.9781217624474374


In [ ]:
final_list = []

In [ ]:
weights_path = 'your/model/weight'
CONV = {
    'f1': ?,
    'f2': ?,
    'ks': ?
}
setup_list = CONV.values().tolist()
model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
csv_list = inference(model, weights_path, val_loader)
record = setup_list + csv_list
final_list.append(csv_list)
pd.DataFrame(final_list, columns=['f1', 'f2', 'ks', 'precision', 'recall', 'ROC', 'f1', 'acc'])

# smote with pca

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DM/smote_dataset'
trainset = pyidata(DATA_PATH, True, True)
testset = pyidata(DATA_PATH, False, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
trainset, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

train_loader = DataLoader(trainset,
              shuffle=True,
              batch_size=32,
              num_workers=4,
              pin_memory=True)
val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)
test_loader = DataLoader(testset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)

In [ ]:

EPOCHS = 700
LR = 0.01
CONV = {
    'f1':16,
    'f2':32,
    'ks':3 
}

model_smote_pca = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_smote_pca.parameters()), lr=LR, momentum=0.9)
CONFIG = {
    'model': model_smote_pca,
    'train_loader':train_loader,
    'val_loader':val_loader, 
    'optimizer':optimizer, 
    'EPOCHS':EPOCHS, 
    'MODELPATH':f'/content/drive/MyDrive/DM/CNN/smote_pca_{LR}_{CONV["f1"]}_{CONV["f2"]}_{CONV["ks"]}.pth'
}
train(**CONFIG)

# downsampling

In [ ]:

DATA_PATH = '/content/drive/MyDrive/DM/downsampling_dataset'
trainset = pyidata(DATA_PATH, True)
testset = pyidata(DATA_PATH, False)
le = len(trainset)
start = int(0.8*le)
end = le - start
trainset, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

train_loader = DataLoader(trainset,
              shuffle=True,
              batch_size=16,
              num_workers=4,
              pin_memory=True)
val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=16,
              num_workers=4,
              pin_memory=True)
test_loader = DataLoader(testset,
              shuffle=False,
              batch_size=16,
              num_workers=4,
              pin_memory=True)

train x shape: (1406, 50)
train y shape: (1406,)
test x shape: (489, 50)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

EPOCHS = 700
LR = 0.01
CONV = {
    'f1':32,#try 32
    'f2':32,
    'ks':9 
}

model_rs = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
# model_rs.load_state_dict(torch.load('/content/drive/MyDrive/DM/CNN/rs_0.01_16_32_3(0.7376).pth'))
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_rs.parameters()), lr=LR, momentum=0.9)
CONFIG = {
    'model': model_rs,
    'train_loader':train_loader,
    'val_loader':val_loader, 
    'optimizer':optimizer, 
    'EPOCHS':EPOCHS, 
    'MODELPATH':f'/content/drive/MyDrive/DM/CNN/rs_{LR}_{CONV["f1"]}_{CONV["f2"]}_{CONV["ks"]}.pth'
}
train(**CONFIG)

  0%|          | 0/71 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 71/71 [00:02<00:00, 29.35it/s]


epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0439   acc: 50.09%
val set=====>loss: 0.0438   acc: 58.16%

save model


100%|██████████| 71/71 [00:02<00:00, 30.21it/s]


epoch 2%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0434   acc: 55.07%
val set=====>loss: 0.0435   acc: 57.80%



100%|██████████| 71/71 [00:02<00:00, 30.35it/s]


...

epoch 1498%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0000   acc: 100.00%
val set=====>loss: 0.2777   acc: 65.25%



100%|██████████| 71/71 [00:02<00:00, 27.01it/s]


epoch 1499%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0000   acc: 100.00%
val set=====>loss: 0.2777   acc: 65.25%



100%|██████████| 71/71 [00:02<00:00, 27.09it/s]


epoch 1500%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0000   acc: 100.00%
val set=====>loss: 0.2780   acc: 64.89%

final model prediction performance on val: 0.6879432624113475


# downsampling with pca

In [ ]:

DATA_PATH = '/content/drive/MyDrive/DM/downsampling_dataset'
trainset = pyidata(DATA_PATH, True, True)
testset = pyidata(DATA_PATH, False, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
trainset, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

train_loader = DataLoader(trainset,
              shuffle=True,
              batch_size=16,
              num_workers=4,
              pin_memory=True)
val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=16,
              num_workers=4,
              pin_memory=True)
test_loader = DataLoader(testset,
              shuffle=False,
              batch_size=16,
              num_workers=4,
              pin_memory=True)

train x shape: (1406, 32)
train y shape: (1406,)
test x shape: (489, 37)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

EPOCHS = 700
LR = 0.01
CONV = {
    'f1':32,
    'f2':32,
    'ks':9
}

model_rs_pca = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
# model_rs_pca.load_state_dict(torch.load('/content/drive/MyDrive/DM/CNN/rs_0.01_16_32_3(0.7376).pth'))
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_rs_pca.parameters()), lr=LR, momentum=0.9)
CONFIG = {
    'model': model_rs_pca,
    'train_loader':train_loader,
    'val_loader':val_loader, 
    'optimizer':optimizer, 
    'EPOCHS':EPOCHS, 
    'MODELPATH':f'/content/drive/MyDrive/DM/CNN/rs_pca_{LR}_{CONV["f1"]}_{CONV["f2"]}_{CONV["ks"]}.pth'
}
train(**CONFIG)

  0%|          | 0/71 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 71/71 [00:00<00:00, 119.60it/s]


epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0438   acc: 51.96%
val set=====>loss: 0.0440   acc: 53.55%

save model


100%|██████████| 71/71 [00:00<00:00, 169.35it/s]


epoch 2%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0435   acc: 54.54%
val set=====>loss: 0.0438   acc: 56.38%

save model


100%|██████████| 71/71 [00:00<00:00, 167.48it/s]


...

epoch 798%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0037   acc: 98.22%
val set=====>loss: 0.3785   acc: 65.96%



100%|██████████| 71/71 [00:00<00:00, 120.54it/s]


epoch 799%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0029   acc: 98.58%
val set=====>loss: 0.3798   acc: 65.60%



100%|██████████| 71/71 [00:00<00:00, 123.22it/s]


epoch 800%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0034   acc: 98.49%
val set=====>loss: 0.3742   acc: 64.89%

final model prediction performance on val: 0.6914893617021277


### we can output all the measurement generated by trained model weight

In [9]:
import os
weight_list = os.listdir('/content/drive/MyDrive/DM/CNN')
weight_list = sorted(weight_list)
weight_list

['rs_0.01_16_32_3(0.7589).pth',
 'rs_0.01_16_32_6(0.6702).pth',
 'rs_0.01_16_32_9(0.6950).pth',
 'rs_0.01_32_32_3(0.7021).pth',
 'rs_0.01_32_32_6(0.6702).pth',
 'rs_0.01_32_32_9(0.6879).pth',
 'rs_pca_0.01_16_32_3(0.6525).pth',
 'rs_pca_0.01_16_32_6(0.6667).pth',
 'rs_pca_0.01_16_32_9(0.7199).pth',
 'rs_pca_0.01_32_32_3(0.6667).pth',
 'rs_pca_0.01_32_32_6(0.?).pth',
 'rs_pca_0.01_32_32_9(0.?).pth',
 'smote_0.01_32_64_3(0.9318).pth',
 'smote_0.01_32_64_6(0.9781).pth',
 'smote_0.01_32_64_9(0.985).pth',
 'smote_0.01_64_64_3(0.9480).pth',
 'smote_0.01_64_64_6(0.9795).pth',
 'smote_0.01_64_64_9(0.9863).pth',
 'smote_pca_0.01_32_64_3(0.9037).pth',
 'smote_pca_0.01_32_64_6(0.9477).pth',
 'smote_pca_0.01_32_64_9(0.9488).pth',
 'smote_pca_0.01_64_64_3(0.9085).pth',
 'smote_pca_0.01_64_64_6(0.9590).pth',
 'smote_pca_0.01_64_64_9(0.9481).pth']

In [10]:
# take all model weight
rs = weight_list[0:6]
rs_pca = weight_list[6:12]
smote = weight_list[12:18]
smote_pca = weight_list[18:24]
weight_list_final = [smote, smote_pca, rs, rs_pca]

### smote

In [12]:
DATA_PATH = '/content/drive/MyDrive/DM/project2/smote_dataset'
trainset = pyidata(DATA_PATH, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
_, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)


train x shape: (82044, 50)
train y shape: (82044,)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
final_list = []
for model_weight in smote:
    conf = model_weight.split('(')[0].split('_')
    CONV = {
        'f1': int(conf[-3]),
        'f2': int(conf[-2]),
        'ks': int(conf[-1])
    }
    
    setup_list = list(CONV.values())
    model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
    csv_list = inference(model, os.path.join('/content/drive/MyDrive/DM/CNN', model_weight), val_loader)
    record = setup_list + csv_list
    final_list.append(record)
    df_smote = pd.DataFrame(final_list, columns=['f1', 'f2', 'ks', 'precision', 'recall', 'ROC', 'f1', 'acc'])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: 

In [14]:
df_smote

,f1,f2,ks,precision,recall,ROC,f1,acc
0,32,64,3,0.925224,0.963458,0.943263,0.943954,0.943141
1,32,64,6,0.986236,0.992888,0.989599,0.989551,0.989579
2,32,64,9,0.994847,0.994237,0.994574,0.994542,0.994576
3,64,64,3,0.939069,0.967627,0.952798,0.953134,0.952709
4,64,64,6,0.982290,0.993010,0.987661,0.987621,0.987629
5,64,64,9,0.994363,0.995095,0.994761,0.994729,0.994759


In [ ]:
df_smote.to_csv('/content/drive/MyDrive/DM/CNN_result/CNN_result_smote.csv')

### smote pca

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DM/smote_dataset'
trainset = pyidata(DATA_PATH, True, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
_, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)


train x shape: (82044, 31)
train y shape: (82044,)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
final_list = []
for model_weight in smote_pca:
    conf = model_weight.split('(')[0].split('_')
    CONV = {
        'f1': int(conf[-3]),
        'f2': int(conf[-2]),
        'ks': int(conf[-1])
    }
    
    setup_list = list(CONV.values())
    model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
    csv_list = inference(model, os.path.join('/content/drive/MyDrive/DM/CNN', model_weight), val_loader)
    record = setup_list + csv_list
    final_list.append(record)
    df_smote_pca = pd.DataFrame(final_list, columns=['f1', 'f2', 'ks', 'precision', 'recall', 'ROC', 'f1', 'acc'])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: 

In [ ]:
df_smote_pca

,f1,f2,ks,precision,recall,ROC,f1,acc
0,32,64,3,0.879665,0.960840,0.916432,0.918463,0.915839
1,32,64,6,0.948372,0.982582,0.965251,0.965174,0.965019
2,32,64,9,0.949693,0.974799,0.962261,0.962082,0.962094
3,64,64,3,0.895331,0.949969,0.920919,0.921841,0.920531
4,64,64,6,0.968514,0.984188,0.976518,0.976288,0.976415
5,64,64,9,0.944768,0.982582,0.963326,0.963304,0.963069


In [ ]:
df_smote_pca.to_csv('/content/drive/MyDrive/DM/CNN_result/CNN_result_smote_pca.csv')

### randomsampling

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DM/downsampling_dataset'
trainset = pyidata(DATA_PATH, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
_, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)


train x shape: (1406, 50)
train y shape: (1406,)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
final_list = []
for model_weight in rs:
    conf = model_weight.split('(')[0].split('_')
    CONV = {
        'f1': int(conf[-3]),
        'f2': int(conf[-2]),
        'ks': int(conf[-1])
    }
    
    setup_list = list(CONV.values())
    model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
    csv_list = inference(model, os.path.join('/content/drive/MyDrive/DM/CNN', model_weight), val_loader)
    record = setup_list + csv_list
    final_list.append(record)
    df_rs = pd.DataFrame(final_list, columns=['f1', 'f2', 'ks', 'precision', 'recall', 'ROC', 'f1', 'acc'])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: 

In [ ]:
df_rs

,f1,f2,ks,precision,recall,ROC,f1,acc
0,16,32,3,0.735714,0.746377,0.744716,0.741007,0.744681
1,16,32,6,0.841060,0.920290,0.876812,0.878893,0.875887
2,16,32,9,0.923611,0.963768,0.943690,0.943262,0.943262
3,32,32,3,0.888889,0.695652,0.806159,0.780488,0.808511
4,32,32,6,0.845638,0.913043,0.876661,0.878049,0.875887
5,32,32,9,0.902098,0.934783,0.918780,0.918149,0.918440


In [ ]:
df_rs.to_csv('/content/drive/MyDrive/DM/CNN_result/CNN_result_rs.csv')

### randomsampling w/ pca

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DM/downsampling_dataset'
trainset = pyidata(DATA_PATH, True, True)
le = len(trainset)
start = int(0.8*le)
end = le - start
_, valset = torch.utils.data.random_split(trainset, lengths=[start, end])

val_loader = DataLoader(valset,
              shuffle=False,
              batch_size=32,
              num_workers=4,
              pin_memory=True)


train x shape: (1406, 32)
train y shape: (1406,)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
final_list = []
for model_weight in rs_pca:
    conf = model_weight.split('(')[0].split('_')
    CONV = {
        'f1': int(conf[-3]),
        'f2': int(conf[-2]),
        'ks': int(conf[-1])
    }
    
    setup_list = list(CONV.values())
    model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])
    csv_list = inference(model, os.path.join('/content/drive/MyDrive/DM/CNN', model_weight), val_loader)
    record = setup_list + csv_list
    final_list.append(record)
    df_rs_pca = pd.DataFrame(final_list, columns=['f1', 'f2', 'ks', 'precision', 'recall', 'ROC', 'f1', 'acc'])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: 

In [ ]:
df_rs_pca

,f1,f2,ks,precision,recall,ROC,f1,acc
0,16,32,3,0.793548,0.884892,0.830558,0.836735,0.829787
1,16,32,6,0.788820,0.913669,0.837953,0.846667,0.836879
2,16,32,9,0.948905,0.935252,0.943150,0.942029,0.943262
3,32,32,3,0.827815,0.899281,0.858731,0.862069,0.858156
4,32,32,6,0.928571,0.841727,0.889395,0.883019,0.890071
5,32,32,9,0.880795,0.956835,0.915480,0.917241,0.914894


In [ ]:
df_rs_pca.to_csv('/content/drive/MyDrive/DM/CNN_result/CNN_result_rs_pca.csv')

### finally, we use smote dataset
#### with f1=64, f2=64, ks=9

In [15]:
DATA_PATH = '/content/drive/MyDrive/DM/project2/smote_dataset'
testset = pyidata(DATA_PATH, False)

test_loader = DataLoader(testset,
              shuffle=False,
              batch_size=512,
              num_workers=4,
              pin_memory=True)


test x shape: (489, 50)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
model_weight = 'smote_0.01_64_64_9(0.9863).pth'
final_list = []
CONV = {
    'f1': 64,
    'f2': 64,
    'ks': 9
}

setup_list = list(CONV.values())
model = Net(f1=CONV['f1'], f2=CONV['f2'], ks=CONV['ks'])

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/DM/CNN', model_weight)))

model.eval()
num = 0
preds = torch.empty(0, device=device)
with torch.no_grad():
    for x, _ in test_loader:
        x = x.to(device).float()
        out = model(x)
        
        num += x.size(0)
        
        pred_ind = torch.max(out, 1).indices.view(-1)

        preds = torch.concat((preds, pred_ind), 0)

print('final prediction of testing datapoint')
print(pred_ind)



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


final prediction of testing datapoint
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  

In [17]:
for name, para in model.named_parameters():
    print(name, para)

model_layers.0.weight Parameter containing:
tensor([[[-3.7350e+00,  2.0650e+00, -5.2876e+00, -1.7536e+00,  3.6158e+00,
          -2.3778e+00,  2.1579e+00, -2.5692e+00,  2.3106e+00]],

        [[-3.5748e+00, -1.6120e+00, -3.4561e+00,  6.3667e-01, -2.9753e-01,
           5.0556e-02,  9.8073e-01, -1.4861e+00, -6.2284e+00]],

        [[-1.9866e+00,  4.4259e-01, -1.5481e-01, -1.2813e+00,  2.8563e+00,
           1.6705e-01, -2.1156e+00, -3.4032e+00, -2.7336e-01]],

        [[ 6.0272e-01, -1.7776e-01,  1.3501e+00, -1.8432e+00, -7.2429e-01,
           5.4286e-01, -6.7824e-01,  4.2297e+00,  6.8620e-01]],

        [[ 1.9505e-01,  7.2825e-01,  1.2959e+00,  7.3452e-01, -3.1479e-01,
           1.3636e+00, -8.2413e+00,  9.1039e-01, -4.9686e+00]],

        [[-3.0516e+00, -1.1146e+00,  1.4278e+00,  1.1359e+00, -1.7675e+00,
           2.1065e+00,  6.9160e-01, -1.8122e+00,  2.1704e+00]],

        [[ 1.3833e+00, -3.4184e+00,  9.2617e-01, -4.2734e-01, -6.8018e-01,
           1.2863e+00,  2.2943e+00, -1.94

In [18]:
model_ = nn.Sequential(*list(model.children())[0])
model_

Sequential(
  (0): Conv1d(1, 64, kernel_size=(9,), stride=(1,))
  (1): ReLU()
  (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv1d(64, 64, kernel_size=(9,), stride=(1,))
  (4): ReLU()
  (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): AdaptiveAvgPool1d(output_size=4)
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=256, out_features=2, bias=True)
)

In [21]:
for x, _ in test_loader:
    x = x.float().cuda()

for i in range(len(model_)):
    x = model_[i](x)
    if i == 7:#output after flatten, shape[batch size, 256]
        out = x
print(out.shape)
print(out)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([489, 256])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.3900, 3.3900],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.1648, 6.7621],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 5.6643, 5.6643],
        [0.0000, 0.0000, 0.2433,  ..., 0.0000, 0.4908, 0.4908]],
       device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
